In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import optuna

In [11]:
from preprocessing import *
data_path = "data/game.csv"
X_train, X_test, y_train, y_test = prep_all(data_path)

X_train = X_train.values.reshape(-1, 1, 136)  # Reshape to (32520, 1, 136)
X_test = X_test.values.reshape(-1, 1, 136)    # Reshape to (N, 1, 136), where N is the number of test samples

train_data = TensorDataset(torch.tensor(X_train), torch.tensor(y_train.to_numpy()))
test_data = TensorDataset(torch.tensor(X_test), torch.tensor(y_test.to_numpy()))

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
valid_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [12]:
class MLPModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLPModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out


In [13]:
input_size = X_train.shape[2]  # The number of features in your preprocessed data
hidden_size = 50
output_size = 2  # Win or loss (binary classification)

model = MLPModel(input_size, hidden_size, output_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [14]:
num_epochs = 1000

for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.squeeze(1).float(), labels.long()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/1000], Loss: 754.5811
Epoch [2/1000], Loss: 6463.7920
Epoch [3/1000], Loss: 4192.3037
Epoch [4/1000], Loss: 2036.9033
Epoch [5/1000], Loss: 3161.6914
Epoch [6/1000], Loss: 1944.3359
Epoch [7/1000], Loss: 4571.6836
Epoch [8/1000], Loss: 3336.6926
Epoch [9/1000], Loss: 219.5071
Epoch [10/1000], Loss: 1158.2913
Epoch [11/1000], Loss: 1285.6168
Epoch [12/1000], Loss: 661.2253
Epoch [13/1000], Loss: 3257.2812
Epoch [14/1000], Loss: 511.8457
Epoch [15/1000], Loss: 4843.9434
Epoch [16/1000], Loss: 4141.8574
Epoch [17/1000], Loss: 4054.7607
Epoch [18/1000], Loss: 972.0283
Epoch [19/1000], Loss: 1366.3325
Epoch [20/1000], Loss: 761.4238
Epoch [21/1000], Loss: 359.7388
Epoch [22/1000], Loss: 2205.2871
Epoch [23/1000], Loss: 927.0166
Epoch [24/1000], Loss: 3265.9917
Epoch [25/1000], Loss: 551.4614
Epoch [26/1000], Loss: 887.0056
Epoch [27/1000], Loss: 253.2947
Epoch [28/1000], Loss: 76.3333
Epoch [29/1000], Loss: 329.2927
Epoch [30/1000], Loss: 610.8616
Epoch [31/1000], Loss: 619.3967
Ep

In [15]:
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in valid_loader:
        inputs, labels = inputs.squeeze(1).float(), labels.long()
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total:.2f}%')


Test Accuracy: 59.58%
